# <p style="text-align:center;">Data 3402: Project 1
## <p style="text-align:center;">Gathering Raw Data
### <p style="text-align:center;">David Wiley

Unlike Data 3401, where projects were disjointed and unrelated, in this course, we will work
a series of projects that build on one another, so that by the end of the semester, you will
have built an engine that will extract raw data from multiple sources (freely available on the
internet), process (parse, clean, transform, merge, etc) that raw data, and store it in a database
for future analysis. In this first project, you will use bash scripting to download, organize,
and store raw data.




## <p style="text-align:center;">The Data and Your Task

Baseball, more than any other sport, has become a business that is driven by data. There are
vast quantities of data freely available through web, so much, in fact, that entire games can
be constructed down to the path, velocity, and rotation of each pitch. While we will not be
dealing with the data to this minute level, we will be gathering data from three major sources.

1. MLB.com
2. Retrosheets.org
3. The Lahman Database

Your task will be to write a multithreaded python script that gathers data from each of these
sources (specified in the following sections) and stores the data according to the scheme that
will be laid out.
Your script should take two arguments:

1. a number of threads, and
2. a directory to store the data.

### 1. MLB.com

We will be particularly interested in the MLB gameday data that is found at https://gd2.mlb.com/components/game/mlb/ . You can navigate the links, but what you need to know is that game data for a specific date MM/DD/YYYY can be found at:

https://gd2.mlb.com/components/game/mlb/year_YYYY/month_MM/day_DD

Notice that I do not include a ’/’ at the end of that URL. Doing so will result in an error.
On this page, you will see a number of links with names of the form **gid_YYYY_MM_DD_***.

Each of these is a directory that stores data for the specified game. For example,
**gid_2018_07_01_anamlb_balmlb_1/** contains data for the game played between the Anaheim Angles and the Baltimore Orioles on 07/01/2018. The ’1’ at the end indicates that this was the first game played between the two on that date. A ’2’ would indicate that the game was the second game in a double-header. The names of these directories are the ”Game IDs” (GID).

Inside this directory, there are three files that you should capture.

1. **players.xml**: contains information
2. **inning all.xml**: contains the result of every event (pitch, pickoff attempt, steal attempt, etc.) that occurred during the game.
3. **inning hit.xml**: contains information about every hit (including location where the ball landed or first touched a fielder).

Your script should do the following:

1. Create a directory called MLB
2. Inside **MLB**, create a directory for each game played between 01/01/2010 and the current date. The name of each game directory should be its GID.
3. In each game directory, you store the three files **players.xml, inning all.xml**, and **inning hit.xml**.

These tasks should be accomplished using a pool of threads. The pool should:

1. take a list of dates (or date objects)
2. extract the GIDs for that date and add them to a queue
3. read the games from the queue and download the game data files.

I know this one was a bit complicated to explain, but don’t worry, the other two sources should be a bit straightforward.

In [3]:
from datetime import date, timedelta
from multiprocessing import Pool, Queue, Process
from requests.exceptions import HTTPError
import requests as rq, os, re, time, wget, sys, queue

def get_GIDs(date):

        try:

            url = 'https://gd2.mlb.com/components/game/mlb/year_'+str(date.year)+'/month_'+str(date.month).zfill(2)+'/day_'+str(date.day).zfill(2)
            r = rq.get(url, timeout=3.05, allow_redirects = True)
            date_string = r.text

            gid_rx = "\s{1}\w+_[0-9]{4}_[0-9]{2}_[0-9]{2}_\w+_\w+_[0-9]"

            match = re.finditer(gid_rx, date_string)

            for m in match:
                q.put((m.group()).strip())
            #print(str(date)+ " done.")
            
        except:
            pass

        
def process_GIDs(q):
    
    while True:
        try:
            gid = q.get_nowait()
            date_rx = '\d{2,4}'
            gid_date = re.findall(date_rx, gid)
            file_path = './MLB/'+gid

            url = 'https://gd2.mlb.com/components/game/mlb/year_'+str(gid_date[0])+'/month_'+str(gid_date[1])+'/day_'+str(gid_date[2])+'/'+gid

            if not (os.path.exists('./MLB/')):
                os.mkdir('./MLB/')
            if not (os.path.exists('./MLB/'+ gid +'/')):
                os.mkdir('./MLB/'+ gid +'/')
            if not (os.path.isfile('./MLB/'+ gid +'/players.xml')):
                wget.download(url+'/players.xml', file_path+'/players.xml')
            if not(os.path.isfile('./MLB/'+ gid +'/inning/inning_hit.xml')):
                wget.download(url+'/inning/inning_hit.xml', file_path +'/inning_hit.xml')
            if not(os.path.isfile('./MLB/'+ gid +'/inning/inning_all.xml')):
                wget.download(url+'/inning/inning_all.xml', file_path +'/inning_all.xml')
                
        except queue.Empty:
            break




if(__name__=="__main__"):
    

    
#     num_threads = sys.argv[1]
#     file_path = sys.argv[2]
    start_time = time.time()
        
    date = date(2019, 9, 1)
    today = date.today()
     
    q = Queue()
    dates = []

        
    while(date <= today):
        dates.append(date)
        date += timedelta(1)
        
        
    print("Getting game IDs...\n")
    

    gid_pool = Pool(20)
    gid_pool.map(get_GIDs, dates)
    gid_pool.close()
    gid_pool.join()
    
    elapse_1 = time.time() - start_time
    print("Finished getting game IDs.")
    print("Time Elapsed: {0} seconds.".format(round(elapse_1, 2)))
    print("Working on processing game IDs...\n")
    
    process_pool = Pool(20, process_GIDs, (q,))
    process_pool.close()
    process_pool.join()
    elapse_2 = time.time() - start_time
    print("Finished processing game IDs.")
    print("Time Elapsed: {0} seconds.".format(round(elapse_2, 2)))


Getting game IDs...

Finished getting game IDs.
Time Elapsed: 1.42 seconds.
Working on processing game IDs...

Finished processing game IDs.
Time Elapsed: 251.87 seconds.


In [43]:
q.qsize()

173

In [44]:
for i in range(q.qsize()):
    print(q.get(i))

gid_2019_09_21_chamlb_detmlb_1
gid_2019_09_21_colmlb_lanmlb_1
gid_2019_09_21_kcamlb_minmlb_1
gid_2019_09_21_nynmlb_cinmlb_1
gid_2019_09_21_phimlb_clemlb_1
gid_2019_09_21_pitmlb_milmlb_1
gid_2019_09_21_seamlb_balmlb_1
gid_2019_09_21_sfnmlb_atlmlb_1
gid_2019_09_21_slnmlb_chnmlb_1
gid_2019_09_21_texmlb_oakmlb_1
gid_2019_09_21_tormlb_nyamlb_1
gid_2019_09_21_wasmlb_miamlb_1
gid_2019_09_20_anamlb_houmlb_1
gid_2019_09_20_arimlb_sdnmlb_1
gid_2019_09_20_bosmlb_tbamlb_1
gid_2019_09_20_chamlb_detmlb_1
gid_2019_09_20_colmlb_lanmlb_1
gid_2019_09_20_kcamlb_minmlb_1
gid_2019_09_20_nynmlb_cinmlb_1
gid_2019_09_20_phimlb_clemlb_1
gid_2019_09_20_pitmlb_milmlb_1
gid_2019_09_20_seamlb_balmlb_1
gid_2019_09_20_sfnmlb_atlmlb_1
gid_2019_09_20_slnmlb_chnmlb_1
gid_2019_09_20_texmlb_oakmlb_1
gid_2019_09_20_tormlb_nyamlb_1
gid_2019_09_20_wasmlb_miamlb_1
gid_2019_09_23_balmlb_tormlb_1
gid_2019_09_23_bosmlb_tbamlb_1
gid_2019_09_23_miamlb_nynmlb_1
gid_2019_09_23_phimlb_wasmlb_1
gid_2019_09_23_slnmlb_arimlb_1
gid_2019

### 2. Retrosheet.org

According to the Society for American Baseball Research (SABR), this website contains data that was compiled by ”a small army of volunteers, combing historical sources to try to re-create the play-by-play of every game in baseball history and digitizing it for download and analysis.” While there are a lot of compiled statistics on the site, we’re primarily interested in the play-by-play event data (as we were with the MLB.com data).

If you navigate to the page https://www.retrosheet.org/game.htm, you will find the following.

1. A link to a listing of players, coaches, managers, and umpires (past and present). This data will be needed for decoding player IDs in the data.
2. Links for regular season event files. For each year, there is a link to a zipped file containing three types of data.
    1. Event data for each MLB team for that year (.eva and .evn extensions)
    2. Roster data for each MLB team for that year (.ros extension)
    3. A file named TeamXXXX (where XXXX is the year) that contains a list of MLB teams in year XXXX. This data will be needed for decoding team IDs in the data (among other things).
3. Links for post-season event file. Again, for each year there is a link to a zipped file containing the same three types of data. You will need to keep the event files, but the team and roster data will have already been acquired from the regular season files.

Your script should do the following.

1. Create a directory called Retrosheets.
2. Inside Retrosheets,
    1. download the list of players, coaches, managers, and umpires to a file called player.csv and
    2. create three directories called Events, Teams, and Rosters.
3. For each year between 1900 and 2019, download and unzip the regular season event data and distribute the event, roster, and team files to the appropriate directories.
4. For each year between 1900 and 2019, download and unzip the post-season event data and copy the event files to the appropriate directory. You do not need to keep the Team and Roster data since that will already have been acquired from the regular season download.


In [7]:
from zipfile import ZipFile

if not (os.path.exists('./Retrosheets/')):
    os.mkdir('./Retrosheets/')

os.chdir('./Retrosheets')
wget.download('https://www.retrosheet.org/retroID.htm', 'player.csv')

if not (os.path.exists('./Events/')):
    os.mkdir('./Events/')
if not (os.path.exists('./Teams/')):
    os.mkdir('./Teams/')
if not (os.path.exists('./Rosters/')):
    os.mkdir('./Rosters/')

for year in range(1900, 2020):
    
    try:
        wget.download('https://www.retrosheet.org/events/'+str(year)+'eve.zip')
        
        zf = ZipFile(str(year)+'eve.zip', 'r')
        zf.extractall()
        zf.close()

        r_event = '\d+\w+.EVN'
        r_rost = '\d+\w+.ROS'
        r_team = 'TEAM\d+'
        r_eva = '\d+\w+.EVA'
        r_zip = '\d+\w+.zip'
        r_eda = '\d+\w+.EDA'
        r_edn = '\d+\w+.EDN'


        for file in os.listdir():
            if(re.search(r_event, file)):
                os.rename(file, './Events/'+str(file))
            if(re.search(r_rost, file)):
                os.rename(file, './Rosters/'+str(file))
            if(re.search(r_team, file)):
                os.rename(file, './Teams/'+str(file))
            elif(re.search(r_eva, file) or re.search(r_zip, file) or re.search(r_eda, file) or re.search(r_edn, file)):
                os.remove(file)
    except:
        pass
    
print('The current working directory was: {}'.format(os.getcwd()))
os.chdir('/home/david/Documents/2019Fall/Data3402/Projects/Project_1')
print('Now the current working directory is: {}'.format(os.getcwd()))


The current working directory was: /home/david/Documents/2019Fall/Data3402/Projects/Project_1/Retrosheets
Now the current working directory is: /home/david/Documents/2019Fall/Data3402/Projects/Project_1


2017
2018
2019


### 3. The Lahman Database
The Lahman Database contains a large number of statistics for every MLB player aggregated by year. You won’t be able to determine if Jose Altuve got a hit in the 6th inning of the Astros’ 2018 season opener, but you will be able to find out his batting average, RBI count, etc. for the 2018 season.

Your script should do the following:

1. Create a directory called Lahman.
2. Download the 2017 Lahman database in CSV format from http://www.seanlahman.com/baseball-archive/statistics
3. Unzip the archive into Lahman




In [8]:
if not (os.path.exists('./Lahman/')):
    os.mkdir('./Lahman/')

os.chdir('./Lahman/')

print('Downloading the file...\n')
wget.download('http://seanlahman.com/files/database/baseballdatabank-master_2018-03-28.zip')

print('Extracting the file to Lahman...\n')
file = ZipFile('baseballdatabank-master_2018-03-28.zip', 'r')
file.extractall()
file.close()

print('Removing unecessary zip file...\n')
os.remove('baseballdatabank-master_2018-03-28.zip')

print('The current working directory was: {}'.format(os.getcwd()))
os.chdir('/home/david/Documents/2019Fall/Data3402/Projects/Project_1')
print('Now the current working directory is: {}'.format(os.getcwd()))



Extracting the file to Lahman...

Removing unecessary zip file...

The current working directory was: /home/david/Documents/2019Fall/Data3402/Projects/Project_1/Lahman
Now the current working directory is: /home/david/Documents/2019Fall/Data3402/Projects/Project_1


### 4. Miscellanea

And that’s the project. Your script should be able to be run as a cron job once daily to pick up any newly released data. It should not just re-download all of the data every time it runs. It should check each date, determine if any data is missing, and only download what is needed. Upload your completed scripts to blackboard by 11:59pm Thursday, 10/10/19.

In [99]:
print('The current working directory was: {}'.format(os.getcwd()))
os.chdir('/home/david/Documents/2019Fall/Data3402/Projects/Project_1')
print('Now the current working directory is: {}'.format(os.getcwd()))

The current working directory is: /home/david/Documents/2019Fall/Data3402/Projects/Project_1/Lahman
Now the current working directory is: /home/david/Documents/2019Fall/Data3402/Projects/Project_1


In [9]:
import os, wget
from zipfile import ZipFile
from pathlib import Path

if (__name__ == "__main__"):

    if not (os.path.exists('./Lahman/')):
        os.mkdir('./Lahman/')
    
    os.chdir('./Lahman/')
    
    print('Downloading the file...\n')
    wget.download('http://seanlahman.com/files/database/baseballdatabank-master_2018-03-28.zip')
    
    print('Extracting the file to Lahman...\n')
    file = ZipFile('baseballdatabank-master_2018-03-28.zip', 'r')
    file.extractall()
    file.close()
    
    print('Removing unecessary zip file...\n')
    os.remove('baseballdatabank-master_2018-03-28.zip')
    
    print('\nThe file was downloaded to: \n{}'.format(os.getcwd()))
    os.chdir(Path(os.getcwd()).parent)
    print('Now the current working directory is: {}'.format(os.getcwd()))


Extracting the file to Lahman...

Removing unecessary zip file...


The file was downloaded to: 
/home/david/Documents/2019Fall/Data3402/Projects/Project_1/Lahman
Now the current working directory is: /home/david/Documents/2019Fall/Data3402/Projects/Project_1
